<img src='../frameworks/img/EU-Copernicus-EUM-WEKEO_banner_logo.png' align='right' width='75%'></img>

<a href="../Index.ipynb"><< Index</a>
<br>
<a href="./2_2_SLSTR_advanced_data_access_hda.ipynb">SLSTR advanced data access; WEkEO HDA >></a>

<font color="#138D75">**Copernicus Marine Training Service**</font> <br>
**Copyright:** 2022 EUMETSAT <br>
**License:** MIT

<div class="alert alert-block alert-success">
<h3>Learn SLSTR: Advanced</h3></div>

<div class="alert alert-block alert-warning">
    
<b>PREREQUISITES </b>
    
The following modules are prerequisites for this notebook:
- **None**

It is recommended to go through these modules before you start with this module.
- **<a href="../1_SLSTR_introductory/1_1_SLSTR_data_access.ipynb">1_1_SLSTR_data_access</a>**

</div>
<hr>

# 2.1 SLSTR advanced data access via the Data Store using eumdac

### Data used

| Product Description  | Data Store collection ID|  WEkEO HDA ID | Product Navigator |
|:--------------------:|:-----------------------:|:-------------:|:-----------------:|
| Sentinel-3 SLSTR level-2 (NTC) | EO:EUM:DAT:0412 | EO:EUM:DAT:SENTINEL-3:SL_2_WST___ | [link](https://navigator.eumetsat.int/product/EO:EUM:DAT:SENTINEL-3:SL_2_SST?query=SLSTR&s=advanced) |

### Learning outcomes

At the end of this notebook you will know;
* How to refine your <font color="#138D75">**searches**</font> for SLSTR products in the EUMETSAT Data Store using the `eumdac client`
* How to <font color="#138D75">**download**</font> components of products

### Outline

...blurb

<div class="alert alert-info" role="alert">

## <a id='TOC_TOP'></a>Contents

</div>
    
1. [Example 1: Filter by collections](#section1)
1. [Example 2: Filter by time](#section2)
1. [Example 3: Filter by space and time](#section3)
1. [Example 4: Download by component](#section4)

<hr>

In [1]:
import os
import json
import datetime
import shutil
import eumdac
import inspect
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import eumartools
from shapely import geometry, vectorized

# Create a download directory for our SLSTR products
download_dir = os.path.join(os.getcwd(), "products")
os.makedirs(download_dir, exist_ok=True)

Authentication...

In [2]:
# load credentials
with open(os.path.join(os.path.expanduser("~"),'.eumdac_credentials')) as json_file:
    credentials = json.load(json_file)
    token = eumdac.AccessToken((credentials['consumer_key'], credentials['consumer_secret']))
    print(f"This token '{token}' expires {token.expiration}")

# create data store object
datastore = eumdac.DataStore(token)

# set collection ID for SLSTR L2 WST
collectionID = 'EO:EUM:DAT:0412'

This token '441584ef-c36b-33ff-b3eb-64386058bc89' expires 2022-02-20 14:29:13.549150


<div class="alert alert-info" role="alert">

## <a id='section1'></a>Example 1: Filtering by collection
[Back to top](#TOC_TOP)

</div>

In [3]:
# Use collection ID
selected_collection = datastore.get_collection(collectionID)
print(selected_collection.title)

SLSTR Level 2 Sea Surface Temperature (SST) - Sentinel-3


<div class="alert alert-info" role="alert">

## <a id='section2'></a>Example 2: Filtering by time
[Back to top](#TOC_TOP)

</div>

In [4]:
# Get the latest product in a collection
selected_collection = datastore.get_collection(collectionID)
latest = selected_collection.search().first()
print(latest)

S3B_SL_2_WST____20220220T103725_20220220T104025_20220220T123206_0179_063_008_3060_MAR_O_NR_003.SEN3


In [5]:
# time filter the collection for products
start = datetime.datetime(2022, 1, 23, 16, 0)
end = datetime.datetime(2022, 1, 23, 16, 15)
products = selected_collection.search(dtstart=start, dtend=end)

for product in products:
    print(product)

S3A_SL_2_WST____20220123T161301_20220123T161601_20220123T183514_0179_081_140_1080_MAR_O_NR_003.SEN3
S3B_SL_2_WST____20220123T161232_20220123T161532_20220123T181246_0179_061_382_3420_MAR_O_NR_003.SEN3
S3A_SL_2_WST____20220123T161001_20220123T161301_20220123T182859_0179_081_140_0900_MAR_O_NR_003.SEN3
S3B_SL_2_WST____20220123T160932_20220123T161232_20220123T181244_0179_061_382_3240_MAR_O_NR_003.SEN3
S3A_SL_2_WST____20220123T160701_20220123T161001_20220123T182808_0179_081_140_0720_MAR_O_NR_003.SEN3
S3B_SL_2_WST____20220123T160632_20220123T160932_20220123T181325_0180_061_382_3060_MAR_O_NR_003.SEN3
S3A_SL_2_WST____20220123T160401_20220123T160701_20220123T182806_0179_081_140_0540_MAR_O_NR_003.SEN3
S3B_SL_2_WST____20220123T160332_20220123T160632_20220123T175723_0179_061_382_2880_MAR_O_NR_003.SEN3
S3A_SL_2_WST____20220123T160101_20220123T160401_20220123T170742_0180_081_140_0360_MAR_O_NR_003.SEN3
S3B_SL_2_WST____20220123T160032_20220123T160332_20220123T175719_0179_061_382_2700_MAR_O_NR_003.SEN3


<div class="alert alert-info" role="alert">

## <a id='section3'></a>Example 3: Filtering by space and time
[Back to top](#TOC_TOP)

</div>

In [6]:
# space/time filter the collection for products
selected_collection = datastore.get_collection(collectionID)
start = datetime.datetime(2022, 1, 23)
end = datetime.datetime(2022, 1, 24)
roi = [[-1.0, -1.0], [4.0, -4.0], [8.0, -2.0], [9.0, 2.0], [6.0, 4.0], [1.0, 5.0], [-1.0, -1.0]]

products = selected_collection.search(
    geo='POLYGON(({}))'.format(','.join(["{} {}".format(*coord) for coord in roi])),
    dtstart=start, 
    dtend=end)

for product in products:
    print(product)

S3A_SL_2_WST____20220123T223659_20220123T223858_20220123T235901_0119_081_143_5940_MAR_O_NR_003.SEN3
S3A_SL_2_WST____20220123T221341_20220123T235440_20220125T085757_6059_081_143______MAR_O_NT_003.SEN3
S3B_SL_2_WST____20220123T215929_20220123T220229_20220123T231323_0179_062_001_0000_MAR_O_NR_003.SEN3
S3B_SL_2_WST____20220123T215730_20220123T215929_20220123T231351_0119_061_385_5940_MAR_O_NR_003.SEN3
S3B_SL_2_WST____20220123T213412_20220123T231511_20220125T083749_6059_061_385______MAR_O_NT_003.SEN3
S3A_SL_2_WST____20220123T100205_20220123T100505_20220123T121359_0179_081_136_3060_MAR_O_NR_003.SEN3
S3A_SL_2_WST____20220123T095905_20220123T100205_20220123T121357_0179_081_136_2880_MAR_O_NR_003.SEN3
S3B_SL_2_WST____20220123T092235_20220123T092535_20220123T111720_0179_061_378_3060_MAR_O_NR_003.SEN3
S3B_SL_2_WST____20220123T091935_20220123T092235_20220123T111718_0179_061_378_2880_MAR_O_NR_003.SEN3
S3A_SL_2_WST____20220123T084547_20220123T102647_20220124T201526_6059_081_135______MAR_O_NT_003.SEN3


<div class="alert alert-info" role="alert">

## <a id='section4'></a>Example 4: Downloading by component
[Back to top](#TOC_TOP)

</div>

In [7]:
# Get the latest product in a collection
selected_collection = datastore.get_collection(collectionID)
latest = selected_collection.search().first()

for entry in latest.entries:
    if 'xfdumanifest.xml' in entry:
        with latest.open(entry=entry) as fsrc, open(os.path.join(os.getcwd(), 'products',fsrc.name),
                                                    mode='wb') as fdst:
            print(f'Downloading {fsrc.name}.')
            shutil.copyfileobj(fsrc, fdst)
            print(f'Download of file {fsrc.name} finished.')

Download of file xfdumanifest.xml finished.


<hr>
<a href="../Index.ipynb"><< Index</a>
<br>
<a href="./2_2_SLSTR_advanced_data_access_hda.ipynb">SLSTR advanced data access; WEkEO HDA >></a>
<hr>
<a href="https://gitlab.eumetsat.int/eumetlab/ocean">View on GitLab</a> | <a href="https://training.eumetsat.int/">EUMETSAT Training</a> | <a href=mailto:ops@eumetsat.int>Contact helpdesk for support </a> | <a href=mailto:Copernicus.training@eumetsat.int>Contact our training team to collaborate on and reuse this material</a></span></p>